**Mounting The Drive**

---



In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


**Specifying Path:**

---

DIR is the path to the directory of the unclean dataset as located in the Drive.
The unclean dataset is stored as newsSpace.

In [0]:
DIR = '/content/drive/My Drive/OurDataset/'

In [0]:
finput = open(DIR + 'newsSpace');

**Cleaning the Dataset:**

---
Steps Followed:


1.   Lines ending with a "\" were concatenated to the string clean_txt, after removing said "\", with a space in between, to form a single line.
2. Lines ending with neither "\N" nor "\" are simply concatenated into clean_txt.
3.   If a line ended with a "\N", it is concatenated with with clean_txt. After which it is subsequently cleaned by:



    *   Replacing and removing HTML encodings
    *   Removing HTML tags
    *   Removing stray "\   \" and "\N"
    *   Removing urls
    *   Removing the time format
    *   Removing stray numerals

4. Data is then split using "\t" as the delimiter.
5. Out this split data we obtained the values for class, title and description.
6. 30,000 unique titles of each class are extracted to populate train.csv.
1900 unique titles of each class are extracted to populate test.csv.
As of now they are stored in their respective dictionary data structure.























In [0]:
import re

txt = finput.readline()
clean_txt = ""

i = 0
j = 0

business = 0
sci = 0
world = 0
sports = 0

uniqueTitles = set()
title = list()
category = list()
description = list()

test_title = list()
test_category = list()
test_description = list()

while txt:
  # print(txt)
  txt = txt.strip()
  # if(i > 783262):
  #   print(txt[-1])
  if len(txt) == 0:
    x = 1;
  elif txt[-1] == '\\':
    txt = txt.rstrip('\\')
    txt = txt.lstrip()
    if len(txt) > 0:
      clean_txt = clean_txt + " " + txt    

  elif txt[-1] == 'N' and txt[-2] == '\\':
    txt = txt.rstrip('\\N ') 
    txt = txt.lstrip()
    clean_txt = clean_txt + " " +txt
    # remove html encodings
    clean_txt = re.sub(r"&#39;", '\'', clean_txt)
    clean_txt = re.sub(r"&#36;", '$', clean_txt)
    clean_txt = re.sub(r"&#37;", '%', clean_txt)
    clean_txt = re.sub(r"&#\d+;", '', clean_txt)
    
    # remove html tags
    clean_txt = re.sub(r"<b>...</b>", '', clean_txt)
    clean_txt = re.sub(r"<\w+>", '', clean_txt)
    clean_txt = re.sub(r"</\w+>", '', clean_txt)
    clean_txt = re.sub(r"<img src.+/>", '', clean_txt)
    clean_txt = re.sub(r"<a href=\"", '', clean_txt)
    clean_txt = re.sub(r"<br/>", ' ', clean_txt)
    # Remove stray \  \ and \N
    clean_txt = re.sub(r"\\\t\\", '', clean_txt)
    clean_txt = re.sub(r"\\N", '', clean_txt)

    # remove url column
    clean_txt = re.sub(r"http\S+", '', clean_txt)
    clean_txt = re.sub(r" Globe -- Business News", '', clean_txt)
    clean_txt = re.sub(r" Globe -- Sports News", '', clean_txt)
    clean_txt = re.sub(r" Globe -- World News", '', clean_txt)

    # remove time column
    clean_txt = clean_txt.rstrip() 
    clean_txt = re.sub(r"\d\d\d\d-\d\d-\d\d\s\d\d:\d\d:\d\d", '', clean_txt)

    #remove number column
    clean_txt = clean_txt.rstrip('\t ') 
    clean_txt = re.sub(r"\d$", '', clean_txt)

    #removing all boundary whitespaces
    clean_txt = clean_txt.strip() 


    # if( i < 20):
    #   print(clean_txt)
    
    columns = re.split(r"\t+", clean_txt);

    n = len(columns)
    
    if n >= 4:
      if not(columns[1] in uniqueTitles):
        m = 4
        des = ""
        while m < n:
          if(len(columns[m].strip()) > 0):
            des = des + columns[m].strip() + " ";
          m += 1
        des.strip('\t ')

        if(columns[3] == 'Business' and business < 31900 and len(des) > 0):
          if(business < 30000):
            title.append(columns[1].strip())
            category.append(3)
            description.append(des)
          elif(business >= 30000 and business < 31900):
            test_title.append(columns[1].strip())
            test_category.append(3)
            test_description.append(des)

          business += 1
        elif(columns[3] == 'Sci/Tech' and sci < 31900 and len(des) > 0):
          if(sci < 30000):
            title.append(columns[1])
            category.append(4)
            description.append(des)
          elif(sci >= 30000 and sci < 31900):
            test_title.append(columns[1].strip())
            test_category.append(4)
            test_description.append(des)
          
          sci += 1
        elif(columns[3] == 'World' and world < 31900 and len(des) > 0):
          if(world < 30000):
            title.append(columns[1])
            category.append(1)
            description.append(des)
          elif(world >= 30000 and world < 31900):
            test_title.append(columns[1].strip())
            test_category.append(1)
            test_description.append(des)

          world += 1;
        elif(columns[3] == 'Sports' and sports < 31900 and len(des) > 0):
          if(sports < 30000):
            title.append(columns[1])
            category.append(2)
            description.append(des)
          elif(sports >= 30000 and sports < 31900):
            test_title.append(columns[1].strip())
            test_category.append(2)
            test_description.append(des)

          sports += 1;

        uniqueTitles.add(columns[2])



    clean_txt = ""
  else:
    clean_txt = clean_txt + txt
    
  i = i + 1
  txt = finput.readline()

print(business)
print(sci)
print(sports)
print(world)

31900
31900
31900
31900


**Populating the CSV Files:**

---
 Using the pandas library the dictionaries storing test data and train data are converted to DataFrames and then into test.csv and train.csv respectively.


In [0]:
import pandas as pd

traindata = {
    'Class' : category,
    'Title' : title,
    'Description': description
} 
testdata = {
    'Class' : test_category,
    'Title' : test_title,
    'Description': test_description
} 

traindf = pd.DataFrame(traindata)
traindf.to_csv(DIR + 'train.csv', header = False, index = False)

testdf = pd.DataFrame(testdata)
testdf.to_csv(DIR + 'test.csv', header = False, index = False)

print("Done")

Done
